# Customer Retention Strategy - Fitness App
**TripleTen Data Analysis Bootcamp**

Author: **Brayan Piñeros**

---

## Objective

(Brief project objective — replace with final sentence before upload)


# Análisis de comportamiento de usuarios, ventas y marketing

## Sección 1. Visitas

### ¿Cuántas personas usan la aplicación cada día, semana y mes?


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
pd.set_option('display.float_format', lambda x: '%.2f' % x)

VISITS_PATH = '/datasets/visits_log_us.csv'
ORDERS_PATH = '/datasets/orders_log_us.csv'
COSTS_PATH  = '/datasets/costs_us.csv'


visits = pd.read_csv(VISITS_PATH)
orders = pd.read_csv(ORDERS_PATH)
costs  = pd.read_csv(COSTS_PATH)


visits['Start Ts'] = pd.to_datetime(visits['Start Ts'], errors='coerce')
visits['End Ts']   = pd.to_datetime(visits['End Ts'], errors='coerce')
orders['Buy Ts']   = pd.to_datetime(orders['Buy Ts'], errors='coerce')
costs['dt']        = pd.to_datetime(costs['dt'], errors='coerce')


if 'Source Id' in visits.columns:
    visits = visits.rename(columns={'Source Id':'source_id'})
else:
    visits = visits.rename(columns={c:c for c in visits.columns})


visits['Uid'] = visits['Uid'].astype(str)
visits['Device'] = visits['Device'].astype('category')
visits['source_id'] = visits['source_id'].astype(str)

orders['Uid'] = orders['Uid'].astype(str)
orders['Revenue'] = pd.to_numeric(orders['Revenue'], errors='coerce')

costs['source_id'] = costs['source_id'].astype(str)
costs['costs'] = pd.to_numeric(costs['costs'], errors='coerce')


print('Shapes:', visits.shape, orders.shape, costs.shape)
print(visits[['Start Ts','End Ts']].info())


In [ ]:
visits_daily   = visits.groupby(visits['Start Ts'].dt.date).size().rename('visits_count').reset_index()
visits_weekly  = visits.groupby(visits['Start Ts'].dt.to_period('W')).size().rename('visits_count').reset_index()
visits_monthly = visits.groupby(visits['Start Ts'].dt.to_period('M')).size().rename('visits_count').reset_index()


display(visits_daily.head(20))

### ¿Cuántas sesiones hay por día?

In [ ]:
sessions_per_day = visits_daily.rename(columns={'Start Ts':'date'})  

plt.figure(figsize=(12,4))
plt.plot(pd.to_datetime(visits_daily['Start Ts']), visits_daily['visits_count'], marker='o', linewidth=0.7)
plt.title('Visitas (sesiones) por día')
plt.xlabel('Fecha')
plt.ylabel('Número de sesiones')
plt.grid(alpha=0.25)
plt.tight_layout()
plt.show()

**Interpretación:**  
Podemos observar los picos de tráfico y los días con mayor uso. Esto puede ayudar a definir cuándo lanzar campañas publicitarias o promociones, optimizando el impacto de las inversiones en marketing.


### ¿Cuál es la duración de cada sesión?

In [ ]:

visits_dur = visits.dropna(subset=['Start Ts','End Ts']).copy()
visits_dur['session_minutes'] = (visits_dur['End Ts'] - visits_dur['Start Ts']).dt.total_seconds() / 60.0


print(visits_dur['session_minutes'].describe())


plt.figure(figsize=(8,4))
plt.hist(visits_dur['session_minutes'].clip(0,60), bins=40) 
plt.title('Distribución de la duración de sesión (min, truncada a 60min)')
plt.xlabel('Duración sesión (min)')
plt.ylabel('Cantidad sesiones')
plt.tight_layout()
plt.show()

### ¿Con qué frecuencia los usuarios regresan?

In [ ]:
sessions_per_user = visits.groupby('Uid').size().rename('sessions_count').reset_index()

mean_sessions_per_user = sessions_per_user['sessions_count'].mean()
median_sessions_per_user = sessions_per_user['sessions_count'].median()
repeat_user_rate = (sessions_per_user['sessions_count'] > 1).mean()

print('Mean sessions per user:', round(mean_sessions_per_user,2))
print('Median sessions per user:', median_sessions_per_user)
print('Repeat user rate:', round(repeat_user_rate,3))


plt.figure(figsize=(8,4))
vals = sessions_per_user['sessions_count'].clip(0,10)
plt.hist(vals, bins=range(0,12))
plt.title('Distribución: sesiones por usuario (valores mayores a 10 agrupados en 10)')
plt.xlabel('Número de sesiones por usuario (cap 10)')
plt.ylabel('Usuarios')
plt.tight_layout()
plt.show()

**Interpretación :**
- `mean_sessions_per_user` muestra la media de sesiones por usuario en el periodo analizado.
- `median_sessions_per_user` reduce sensibilidad a outliers.
- `repeat_user_rate` es la fracción de usuarios que volvieron al menos una vez.

# VENTAS

### ¿Cuándo empieza la gente a comprar? (Tiempo entre registro y conversión)

In [ ]:
first_visit = visits.groupby('Uid')['Start Ts'].min().rename('first_visit').reset_index()
first_order = orders.groupby('Uid')['Buy Ts'].min().rename('first_order').reset_index()

cohort = first_visit.merge(first_order, on='Uid', how='left')
cohort['conversion_days'] = (cohort['first_order'] - cohort['first_visit']).dt.days

def label_conversion(d):
    if pd.isna(d):
        return 'No conversion'
    if d <= 0:
        return 'Conversion 0d'
    if d == 1:
        return 'Conversion 1d'
    if d == 2:
        return 'Conversion 2d'
    if 3 <= d <= 7:
        return 'Conversion 3-7d'
    if 8 <= d <= 30:
        return 'Conversion 8-30d'
    return '>30d'

cohort['conversion_category'] = cohort['conversion_days'].apply(label_conversion)


conv_counts = cohort['conversion_category'].value_counts().reindex(['No conversion','Conversion 0d','Conversion 1d','Conversion 2d','Conversion 3-7d','Conversion 8-30d','>30d']).fillna(0)
print(conv_counts)


plt.figure(figsize=(8,5))
bars = plt.bar(conv_counts.index, conv_counts.values)
plt.title('Distribución de cohortes: días hasta primera compra')
plt.xlabel('Categoría de conversión')
plt.ylabel('Usuarios')
plt.xticks(rotation=45, ha='right')

for b in bars:
    h = b.get_height()
    plt.annotate(f'{int(h):,}', xy=(b.get_x() + b.get_width()/2, h), xytext=(0,3), textcoords='offset points', ha='center', fontsize=9)
plt.tight_layout()
plt.show()

### ¿Cuántos pedidos hacen durante un período de tiempo dado?

In [ ]:
orders_per_day = orders.groupby(orders['Buy Ts'].dt.date).size().rename('orders_count').reset_index()
orders_per_month = orders.groupby(orders['Buy Ts'].dt.to_period('M')).size().rename('orders_count').reset_index()


plt.figure(figsize=(10,4))
plt.plot(pd.to_datetime(orders_per_month['Buy Ts'].astype(str)), orders_per_month['orders_count'], marker='o')
plt.title('Pedidos por mes')
plt.xlabel('Mes')
plt.ylabel('Número de pedidos')
plt.tight_layout()
plt.show()

### ¿Cuál es el tamaño promedio de compra?

In [ ]:

aov = orders['Revenue'].mean()
print('AOV:', round(aov,2))

ltv_per_user = orders.groupby('Uid')['Revenue'].sum().rename('ltv').reset_index()
ltv_mean = ltv_per_user['ltv'].mean()
print('LTV promedio por usuario:', round(ltv_mean,2))


cohort_with_ltv = first_visit.merge(ltv_per_user, on='Uid', how='left')
cohort_with_ltv['cohort_month'] = cohort_with_ltv['first_visit'].dt.to_period('M').astype(str)
ltv_by_cohort = cohort_with_ltv.groupby('cohort_month')['ltv'].agg(['count','mean','sum']).reset_index().rename(columns={'count':'customers','mean':'ltv_mean','sum':'ltv_sum'})
print(ltv_by_cohort.sort_values('cohort_month').head(12))

plt.figure(figsize=(10,4))
plt.bar(ltv_by_cohort['cohort_month'], ltv_by_cohort['ltv_mean'])
plt.xticks(rotation=45, ha='right')
plt.title('LTV medio por cohorte (mes de primera visita)')
plt.xlabel('Cohorte (Mes)')
plt.ylabel('LTV medio ($)')
plt.tight_layout()
plt.show()


# MARKETING

### 1. ¿Cuánto dinero se gastó? (Total/por fuente de adquisición/a lo largo del tiempo)

In [ ]:

total_spent = costs['costs'].sum()
spend_by_source = costs.groupby('source_id')['costs'].sum().reset_index().rename(columns={'costs':'total_cost'})
spend_over_time = costs.groupby('dt')['costs'].sum().reset_index().rename(columns={'costs':'daily_spend'})

print('Total marketing spend:', round(total_spent,2))
display(spend_by_source.sort_values('total_cost', ascending=False).head(20))

plt.figure(figsize=(12,4))
plt.plot(spend_over_time['dt'], spend_over_time['daily_spend'], marker='o', linewidth=0.7)
plt.title('Gasto en marketing a lo largo del tiempo')
plt.xlabel('Fecha')
plt.ylabel('Gasto diario ($)')
plt.grid(alpha=0.2)
plt.tight_layout()
plt.show()

### 2. ¿Cuál fue el costo de adquisición de clientes (CAC) por fuente?

In [ ]:
converted = cohort[~cohort['first_order'].isna()][['Uid']].copy()

acquisition = visits.sort_values('Start Ts').groupby('Uid')['source_id'].first().rename('acquisition_source').reset_index()
converted = converted.merge(acquisition, on='Uid', how='left')

customers_by_source = converted.groupby('acquisition_source')['Uid'].nunique().reset_index().rename(columns={'Uid':'customers'})
spend_by_source_renamed = spend_by_source.rename(columns={'source_id':'acquisition_source'})
cac = spend_by_source_renamed.merge(customers_by_source, on='acquisition_source', how='left')
cac['customers'] = cac['customers'].fillna(0).astype(int)

cac['CAC'] = cac.apply(lambda r: (r['total_cost'] / r['customers']) if r['customers']>0 else np.nan, axis=1)
display(cac.sort_values('CAC').head(20))

### 3. ¿Cuán rentables eran las inversiones? (ROMI)

In [ ]:
orders_with_source = orders.merge(acquisition, on='Uid', how='left')
revenue_by_source = orders_with_source.groupby('acquisition_source')['Revenue'].sum().reset_index().rename(columns={'Revenue':'revenue'})

romi = spend_by_source_renamed.merge(revenue_by_source, on='acquisition_source', how='left')
romi['revenue'] = romi['revenue'].fillna(0)
romi['ROMI'] = romi.apply(lambda r: (r['revenue'] - r['total_cost'])/r['total_cost'] if r['total_cost']>0 else np.nan, axis=1)

display(romi.sort_values('ROMI', ascending=False).head(20))

### Visualizaciones comparativas

In [ ]:

def annotate_bars(ax):
    for p in ax.patches:
        h = p.get_height()
        if np.isfinite(h) and h>0:
            ax.annotate(f'{h:,.0f}', (p.get_x()+p.get_width()/2., h), ha='center', va='bottom', fontsize=8, rotation=0, xytext=(0,3), textcoords='offset points')


top_spend = spend_by_source.sort_values('total_cost', ascending=False).head(10)
fig, ax = plt.subplots(figsize=(10,4))
ax.bar(top_spend['source_id'], top_spend['total_cost'])
ax.set_title('Gasto total por fuente (top 10)')
ax.set_xlabel('Fuente')
ax.set_ylabel('Gasto total ($)')
plt.xticks(rotation=45, ha='right')
annotate_bars(ax)
plt.tight_layout()
plt.show()


cac_plot = cac[~cac['CAC'].isna()].sort_values('CAC', ascending=True).head(10)
fig, ax = plt.subplots(figsize=(10,4))
ax.bar(cac_plot['acquisition_source'], cac_plot['CAC'])
ax.set_title('CAC por fuente (top 10 menor CAC)')
ax.set_xlabel('Fuente')
ax.set_ylabel('CAC ($)')
plt.xticks(rotation=45, ha='right')
annotate_bars(ax)
plt.tight_layout()
plt.show()


romi_plot = romi.sort_values('ROMI', ascending=False).head(10)
fig, ax = plt.subplots(figsize=(10,4))
ax.bar(romi_plot['acquisition_source'], romi_plot['ROMI'])
ax.set_title('ROMI por fuente (top 10)')
ax.set_xlabel('Fuente')
ax.set_ylabel('ROMI (ratio)')
plt.xticks(rotation=45, ha='right')
# Anotar romi en formato porcentaje
for p in ax.patches:
    val = p.get_height()
    if np.isfinite(val):
        ax.annotate(f'{val:.2f}', (p.get_x()+p.get_width()/2., val), ha='center', va='bottom', fontsize=8, xytext=(0,3), textcoords='offset points')
plt.tight_layout()
plt.show()

## Paso 3 — Conclusiones y recomendaciones para inversión en marketing